## Imports and Data Load

In [1]:
from utils_v2 import *
from UNet import *

In [2]:
import gc

gc.collect()

0

## Dataset

In [61]:
ds = xr.open_dataset('data_850/2022_850_SA.nc')
ds.load()

bounds = [ds.longitude.min().item(), ds.longitude.max().item(), ds.latitude.min().item(), ds.latitude.max().item()]

print(bounds)

weather_data = WeatherData(ds, window_size=3)
weather_data.subset_data()
weather_data.window_dataset()

[15.555999755859375, 33.05699920654297, -35.13800048828125, -26.886999130249023]


## Model

In [16]:
def build_unet(input_shape, output_channels = 1):
    input_data = Input(input_shape)

    s1, p1 = encoder_block(input_data, 16)
    s2, p2 = encoder_block(p1, 32)
    # s3, p3 = encoder_block(p2, 256)
    # s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p2, 64) #Bridge

    # d1 = decoder_block(b1, s4, 512)
    # d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(b1, s2, 32)
    d4 = decoder_block(d3, s1, 16)

    time_input = Input(shape=(2,))

    outputs = Conv2D(output_channels, (1, 1), activation='sigmoid')(d4)
    
    model = Model(inputs=[input_data, time_input], outputs=outputs, name="U-Net-Forcings")
    return model


## Preprocessing

In [92]:

class WeatherMLModel:
    def __init__(self, model = None, steps = 3):
        """
        Initializes the WeatherMLModel class.

        Parameters:
        - model: A machine learning model (e.g., sklearn model, keras model).
        - data: The input data for training the model.
        - target: The target variable for training the model.
        """
        self.model = model
        self.features = None
        self.targets = None
        self.forcings = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.F_train = None
        self.F_test = None
        self.T_train = None
        self.T_test = None
        self.plot_shape = None
        self.predictions = None

        print('Class setup done...')

    def load_data(self, features, targets, forcings, time_values):
        """
        Loads the input data and target variable.

        Parameters:
        - features: The input data for training the model.
        - targets: The target variable for training the model.
        """

        self.plot_shape = features.shape[1:]

        self.features = features.reshape(features.shape[0], features.shape[2], features.shape[3], features.shape[1]).astype('float32')
        self.targets = targets.reshape(targets.shape[0], targets.shape[2], targets.shape[3], targets.shape[1]).astype('float32')
        self.forcings = forcings
        self.time_values = time_values

        print('Data loaded...')

    def assign_model(self, model):
        self.model = model

        print('Model assigned...')
    
    def split_data(self, test_size=0.2, random_state=42):
        """
        Splits the data into training, validation, and test sets.
        """

        print('Splitting...')
        self.X_train, self.X_test, self.y_train, self.y_test, self.F_train, self.F_test, self.T_train, self.T_test = train_test_split(
            self.features, self.targets, self.forcings, self.time_values,
            test_size= test_size)
        
        print('Shuffling...')
        
        self.X_train, self.y_train, self.F_train, self.T_train = shuffle(self.X_train, self.y_train, self.F_train, self.T_train, random_state=random_state)
        
    def check_model(self):
        self.model.summary()  
        
        print(self.model.predict([self.X_train[0:1], self.F_train[0:1]]).shape)
    
    def train_model(self, patience=10, best_model_name=None, max_epochs=100, val_split = 0.8, return_history=False):
        """
        Trains the machine learning model.
        """
        if best_model_name is None:
            current_time = datetime.now()
            formatted_time = current_time.strftime('%m_%d_%H_%M')

            best_model_name = f'models/{formatted_time}.h5'
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min', verbose=1)
  
        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
                best_model_name,
                monitor='val_loss',
                save_best_only=True,
                mode='min',
                verbose=0
            )
        
        self.model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])
        
        print('Compiled...')

        if val_split != 0:
            split = int(self.X_train.shape[0] * val_split)

            history = self.model.fit([self.X_train[:split], self.F_train[:split]], self.y_train[:split], epochs=max_epochs,
                        validation_data=([self.X_train[split:], self.F_train[split:]], self.y_train[split:]),
                        callbacks=[early_stopping, model_checkpoint])
        else:
            history = self.model.fit([self.X_train, self.F_train], self.y_train, epochs=max_epochs,
                        callbacks=[early_stopping, model_checkpoint])
            
        if return_history:
            return history

    def evaluate_model(self):
        """
        Evaluates the trained model.
        """
        self.predictions = self.model.predict([self.X_test, self.F_test])

        return mean_squared_error(self.y_test.flatten(), self.predictions.flatten(), squared=False)

    def load_model(self, filepath):
        """
        Loads a model from a file.

        Parameters:
        - filepath: The path to the file from which the model will be loaded.
        """
        self.model = tf.keras.models.load_model(filepath)

    def plot_predictions(self, seed, levels=10, frame_rate=10):
        bounds=[15.555999755859375, 33.05699920654297, -35.13800048828125, -26.886999130249023]
        features = self.X_test[seed:seed+1]
        forcings = self.F_test[seed:seed + 1]
        targets = self.y_test[seed:seed + 1].reshape(1, self.y_test.shape[3], self.y_test.shape[1], self.y_test.shape[2])

        prediciton = self.model.predict([features, forcings]).reshape(1, self.y_test.shape[3], self.y_test.shape[1], self.y_test.shape[2])

        vmax = max(prediciton.max(), targets.max())
        vmin = min(prediciton.min(), targets.min())

        fig, axs = plt.subplots(1, 3, figsize=(21, 7), subplot_kw={'projection': ccrs.PlateCarree()})

        for ax in axs:
            ax.set_extent(bounds, crs=ccrs.PlateCarree())
            ax.coastlines()

        fig.subplots_adjust(left=0.05, right=0.95, bottom=0.1, top=0.9, wspace=0.2)

        contour = axs[0].contourf(targets[seed,0], levels=levels, vmin=vmin, vmax=vmax)
        cbar = plt.colorbar(contour, ax=axs[0], shrink=0.3, aspect=10)

        error = targets - prediciton

        vmax_e = error.max()
        vmin_e = error.min()

        contour_e = axs[2].contourf(error[0,1], levels=levels, vmin=vmin_e, vmax=vmax_e, cmap='coolwarm')
        colorbar = plt.colorbar(contour_e, ax=axs[2], shrink=0.3, aspect=10)

        contour_a =axs[1].contourf(prediciton[0,0], levels=levels, vmin=vmin, vmax=vmax)
        colorbar = plt.colorbar(contour_a, ax=axs[1], shrink=0.3, aspect=10)

        def animate(i):
            for ax in axs:
                ax.clear()
                ax.coastlines()

            contour_a = axs[0].contourf(targets[0, i], levels=levels, vmin=vmin, vmax=vmax)
            axs[0].set_title(f'Analysis Wind Speed (m/s)')

            contour_f = axs[1].contourf(prediciton[0, i], levels=levels, vmin=vmin, vmax=vmax)
            axs[1].set_title(f'Forecast Wind Speed (m/s)')

            contour_e = axs[2].contourf(error[0, i], levels=levels, vmin=vmin_e, vmax=vmax_e, cmap='coolwarm')
            axs[2].set_title(f'Error Wind Speed (m/s)')

        interval = 1000 / frame_rate

        ani = FuncAnimation(fig, animate, frames=targets.shape[1], interval=interval)

        plt.close(fig)

        return HTML(ani.to_jshtml())
    

In [93]:
model_class = WeatherMLModel()

features, targets, forcings, time_values = weather_data.return_data()
model_class.load_data(features=features, targets=targets, forcings=forcings, time_values=time_values)
model_class.split_data()

model = build_unet(model_class.X_train.shape[1:], model_class.y_train.shape[-1])
model_class.assign_model(model)

Class setup done...
Data loaded...
Splitting...
Shuffling...
Model assigned...


## Plotting predictions

In [33]:
model_class.train_model(patience=10, max_epochs=10, val_split=0.8)

Compiled...
176/176 [==============================] - 22s 98ms/step - loss: 46.7117 - mean_absolute_error: 5.5954 - val_loss: 45.4044 - val_mean_absolute_error: 5.5034


c:\Users\23603526\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [94]:
model_class.load_model('models/09_11_10_16.h5')

In [95]:
model_class.plot_predictions(0, levels=10, frame_rate=10)

1/1 [==============================] - 0s 286ms/step


In [83]:
model_class.y_test[0:1].reshape(1, model_class.y_test.shape[3], model_class.y_test.shape[1], model_class.y_test.shape[2]).shape

(1, 3, 32, 64)

In [66]:
seed = 0
levels = 10
frame_rate = 10
bounds = [15.555999755859375, 33.05699920654297, -35.13800048828125, -26.886999130249023]

def plot_predictions(self, seed, levels=10, frame_rate=10):
    bounds=[15.555999755859375, 33.05699920654297, -35.13800048828125, -26.886999130249023]
    features = X_test[seed:seed+1]
    forcings = X_forcing_test[seed:seed + 1]
    targets = y_test[seed:seed + 1].reshape(1, 3, 32, 64)

    prediciton = model_class.model.predict([features, forcings]).reshape(1, 3, 32, 64)

    vmax = max(prediciton.max(), targets.max())
    vmin = min(prediciton.min(), targets.min())

    fig, axs = plt.subplots(1, 3, figsize=(21, 7), subplot_kw={'projection': ccrs.PlateCarree()})

    for ax in axs:
        ax.set_extent(bounds, crs=ccrs.PlateCarree())
        ax.coastlines()

    fig.subplots_adjust(left=0.05, right=0.95, bottom=0.1, top=0.9, wspace=0.2)

    contour = axs[0].contourf(targets[seed,0], levels=levels, vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(contour, ax=axs[0], shrink=0.3, aspect=10)

    error = targets - prediciton

    vmax_e = error.max()
    vmin_e = error.min()

    contour_e = axs[2].contourf(error[0,1], levels=levels, vmin=vmin_e, vmax=vmax_e, add_colorbar=False, cmap='coolwarm')
    colorbar = plt.colorbar(contour_e, ax=axs[2], shrink=0.3, aspect=10)

    contour_a =axs[1].contourf(prediciton[0,0], levels=levels, vmin=vmin, vmax=vmax, add_colorbar=False)
    colorbar = plt.colorbar(contour_a, ax=axs[1], shrink=0.3, aspect=10)

    def animate(i):
        for ax in axs:
            ax.clear()
            ax.coastlines()

        contour_a = axs[0].contourf(targets[0, i], levels=levels, vmin=vmin, vmax=vmax, add_colorbar=False)
        axs[0].set_title(f'Analysis Wind Speed (m/s)')

        contour_f = axs[1].contourf(prediciton[0, i], levels=levels, vmin=vmin, vmax=vmax, add_colorbar=False)
        axs[1].set_title(f'Forecast Wind Speed (m/s)')

        contour_e = axs[2].contourf(error[0, i], levels=levels, vmin=vmin_e, vmax=vmax_e, add_colorbar=False, cmap='coolwarm')
        axs[2].set_title(f'Error Wind Speed (m/s)')

    interval = 1000 / frame_rate

    ani = FuncAnimation(fig, animate, frames=targets.shape[1], interval=interval)

    plt.close(fig)

    return HTML(ani.to_jshtml())

1/1 [==============================] - 0s 87ms/step


c:\Users\23603526\AppData\Local\Programs\Python\Python311\Lib\site-packages\cartopy\mpl\geoaxes.py:1642: UserWarning: The following kwargs were not used by contour: 'add_colorbar'
  result = super().contourf(*args, **kwargs)
c:\Users\23603526\AppData\Local\Programs\Python\Python311\Lib\site-packages\cartopy\mpl\geoaxes.py:1642: UserWarning: The following kwargs were not used by contour: 'add_colorbar'
  result = super().contourf(*args, **kwargs)
c:\Users\23603526\AppData\Local\Programs\Python\Python311\Lib\site-packages\cartopy\mpl\geoaxes.py:1642: UserWarning: The following kwargs were not used by contour: 'add_colorbar'
  result = super().contourf(*args, **kwargs)
c:\Users\23603526\AppData\Local\Programs\Python\Python311\Lib\site-packages\cartopy\mpl\geoaxes.py:1642: UserWarning: The following kwargs were not used by contour: 'add_colorbar'
  result = super().contourf(*args, **kwargs)
c:\Users\23603526\AppData\Local\Programs\Python\Python311\Lib\site-packages\cartopy\mpl\geoaxes.py:1